In [112]:
import urllib, string, time
import urllib.parse
from astropy.coordinates import SkyCoord
from astropy import units as u
import pandas as pd
from astropy.time import Time
import numpy as np
from tabulate import tabulate
from astroquery.simbad import Simbad

from bokeh.plotting import output_notebook, figure, show
from bokeh.models import HoverTool, tools,ColumnDataSource
from bokeh.palettes import viridis, inferno
from bokeh.models import HoverTool, tools,ColumnDataSource, Whisker, ColorBar, LinearColorMapper

output_notebook()

Loading BokehJS ...

In [187]:
namecluster = 'NGC 6205'
result_table = Simbad.query_object(namecluster)
print(result_table)
table = result_table[0]


MAIN_ID      RA          DEC      ... COO_WAVELENGTH     COO_BIBCODE    
          "h:m:s"      "d:m:s"    ...                                   
------- ------------ ------------ ... -------------- -------------------
  M  13 16 41 41.634 +36 27 40.75 ...              I 2006AJ....131.1163S


In [188]:
racluster = table['RA']
deccluster = table['DEC']
c = SkyCoord(f'{racluster} {deccluster}', unit=(u.hourangle, u.deg))

ra = c.ra.deg
dec= c.dec.deg
radius = 3.0

# create URL with desired search parameters

url = "https://archive.stsci.edu/hst/search.php?"
url = url + f"&RA={ra}"
url = url + f"&DEC={dec}"
url = url + f"&radius={radius}"
url = url + "&outputformat=CSV"
url = url + "&action=Search"


data = pd.read_csv(url,low_memory=False)

data = data.drop(data.index[0])
data = data[data['Start Time'].apply(lambda x: isinstance(x, str))]

#m4data
#data['YearDay'] = data['Start Time'].map(lambda times: '.'.join(Time(times).yday.split(':',2)[:2]))

#data['Week'] = data['Start Time'].map(lambda times: int(np.floor(float(Time(times).yday.split(':')[1])/7.)))
#data2 = data[data['High-Level Science Products'] == '1']
t =data.groupby(['Target Name','Filters/Gratings','Instrument','Exp Time','Proposal ID']).size().reset_index().rename(columns={0:'ocurrence'})
#t.sort_values(by=['ocurrence'],ascending=False)
print(tabulate(t.sort_values(by=['ocurrence'],ascending=False), tablefmt="pipe", headers="keys"))

|    | Target Name           | Filters/Gratings   | Instrument   |   Exp Time |   Proposal ID |   ocurrence |
|---:|:----------------------|:-------------------|:-------------|-----------:|--------------:|------------:|
| 42 | NGC6205               | F814W              | WFPC2        |    140     |          8278 |          25 |
| 37 | NGC6205               | F555W              | WFPC2        |     80     |          8278 |          25 |
| 41 | NGC6205               | F785LP             | WFPC2        |     26     |          8174 |          12 |
| 35 | NGC6205               | F555W              | WFPC2        |     26     |          8174 |          12 |
| 11 | M13-WFPC2             | F450W              | WFPC2        |    350     |         11233 |           9 |
| 12 | M13-WFPC2             | F814W              | WFPC2        |    100     |         11233 |           9 |
| 43 | NGC6205               | F814W              | WFPC2        |     23     |          8278 |           5 |
|  1 | BIA

In [192]:
#data['MJD'] = data['Start Time'].map(lambda times: Time(times).datetime64)
#data['YearDay'] = data['Start Time'].map(lambda times: '.'.join(Time(times).yday.split(':',2)[:2]))
data['year'] = data['Start Time'].map(lambda times: Time(times).yday.split(':')[0])

filtertofilter = 'F606W'

datacleaner =data[(data['Target Name'] != 'DARK') & (data['Target Name'] != 'BIAS') & (data['Target Name'] != 'EARTH-CALIB')]
dataf555w  = datacleaner[datacleaner['Filters/Gratings'] == filtertofilter]

t2 =dataf555w.groupby(['Target Name','Filters/Gratings','Instrument','Exp Time','year']).size().reset_index().rename(columns={0:'ocurrence'})
#t2 =data.groupby(['Target Name','Filters/Gratings','Instrument']).size().reset_index().rename(columns={0:'ocurrence'})

t2.sort_values(by=['ocurrence'],ascending=False)
print(tabulate(t2.sort_values(by=['ocurrence'],ascending=False), tablefmt="pipe", headers="keys"))


| Target Name   | Filters/Gratings   | Instrument   | Exp Time   | year   | ocurrence   |
||


In [ ]:
dataf555w  = datacleaner[(datacleaner['Filters/Gratings'] == 'F555W')&(datacleaner['Instrument'] == 'WFPC2 ') 
                         &(datacleaner['year'] == '2008')]

timelist = dataf555w['MJD'].tolist()
exptime = dataf555w['Exp Time'].to_numpy(dtype='float')
namesobs = dataf555w['Start Time'].tolist()

datatoplot = {'time':timelist,
       'y':exptime,
        'ones':[1.]*len(exptime),
       'names':namesobs}

source = ColumnDataSource(data=datatoplot)

p = figure(plot_width=900, plot_height=500, title='a',active_drag='pan', active_scroll='wheel_zoom',
          x_axis_type='datetime',y_axis_label='ExpTime (s)',x_axis_label='Date_Obs')


#Tool to get wavelength
hover2 = HoverTool(
        tooltips=[
            ('Date', '(@names)'),
            ("EXPTime",'(@y)')
        ]
    )

        
p.add_tools(hover2)



# add a circle renderer with a size, color, and alpha
#p.circle('time','y', source=source)

p.add_tools(hover2)


colorlist = inferno(len(exptime))

mapper = LinearColorMapper(palette=colorlist, low=exptime.min(), high=exptime.max())
color_bar = ColorBar(color_mapper=mapper, location=(0, 0.5),title='ExpTime')
p.circle('time','ones', source=source,color={'field': 'y', 'transform': mapper})
p.add_layout(color_bar, 'right')
show(p)

In [278]:
dataf555w['Instrument']

25     WFPC  
58     WFC3  
172    WFPC2 
173    WFPC2 
174    WFPC2 
        ...  
375    WFPC2 
376    WFPC2 
377    WFPC2 
378    WFPC2 
379    WFPC2 
Name: Instrument, Length: 161, dtype: object

In [259]:
dataf555w['Exp Time'].to_numpy(dtype='float').min()

2.0